In [1]:
import pandas as pd  #importing pandas
import numpy as np    #importing numpy
df_new= pd.read_csv('/home/manpreet/mps/cc/aclImdb_v1/aclImdb/movie_reviews.csv')   #reading csv
df_new.head()    #print some value from top of df_new


 



,review,sentiment
0,Richard Brooks' The Last Hunt was a film star ...,1
1,"Pat Conroy's autobiographical book ""The Water ...",1
2,The Five Deadly Venoms is easily the most memo...,1
3,An interesting concept turned into carnage...<...,0
4,"Its not Braveheart( thankfully),but it is fine...",1


In [2]:

df_new.sentiment.value_counts()  #this counts the occurance of allsentiments and print the correspondence value

1    25000
0    25000
Name: sentiment, dtype: int64

In [3]:

from sklearn.utils import shuffle
print(df_new.head())
df_new = shuffle(df_new)     #shuffling the values in df_new
print(df_new.head())


                                              review  sentiment
0  Richard Brooks' The Last Hunt was a film star ...          1
1  Pat Conroy's autobiographical book "The Water ...          1
2  The Five Deadly Venoms is easily the most memo...          1
3  An interesting concept turned into carnage...<...          0
4  Its not Braveheart( thankfully),but it is fine...          1
                                                  review  sentiment
36127  UGH!...this is the worst Wrestlemania hands do...          0
7584   Oh how I laughed....this has it all...an Asian...          0
15094  Great voices, lots of adventure and clever dia...          1
49187  Even if you do not typically enjoy documentari...          1
7284   Well, I AM "the target market" & I loved it. F...          1


In [4]:
import re    ##importing regular expression
def preprocessor(text):   #function to remove extra thing from the string for ex emoticons
    
    #we are using regular expression to determine emoticons and other extra things
   text = re.sub('<[^>]*>','',text)     
   emoticons = re.findall('[:;=](?:-)', text)
   text = (re.sub('[\W]+',' ',text.lower())+ ' '.join(emoticons).replace('-',''))
   
   return text


In [5]:
preprocessor("</a>This is# a ;test ::-)!</a>")   #example what above function is doing

'this is a test :'

'this is a test :'

In [6]:
df_new['review'] = df_new['review'].apply(preprocessor)   #applying above defined function to all the reviews and
#then replacing those reviews
df_new.head()

,review,sentiment
36127,ugh this is the worst wrestlemania hands down ...,0
7584,oh how i laughed this has it all an asian whit...,0
15094,great voices lots of adventure and clever dial...,1
49187,even if you do not typically enjoy documentari...,1
7284,well i am the target market i loved it further...,1


In [7]:
df_new.index  =range(50000)
df_new.head()

,review,sentiment
0,ugh this is the worst wrestlemania hands down ...,0
1,oh how i laughed this has it all an asian whit...,0
2,great voices lots of adventure and clever dial...,1
3,even if you do not typically enjoy documentari...,1
4,well i am the target market i loved it further...,1


In [8]:
X_train = df_new.loc[:2500,'review'].values
y_train = df_new.loc[:2500,'sentiment'].values
X_test = df_new.loc[2500:,'review'].values
y_test = df_new.loc[2500:,'sentiment'].values

In [9]:
print(np.bincount(y_test))





# bincount creates the list of counts of index that occur in the list

#for ex a=[1,2,1,3,4,5,5,5]
#index are 0,1,2,3,4,5,6,7 respectively for 1,2,1,3,4,5,5,5

#bincount will count the occurrence of index in self 
# ex below
print("\n\nEXample result\n")
a=np.array([1,2,1,3,4,5,5,5])
print(np.bincount(a))

#occurrence of INDEX is NO.OF TIME in array a
#occurrence of 0 is 0 in array a
#occurrence of 1 is 2 in array a
#occurrence of 2 is 1 in array a
#occurrence of 2 is 2 in array a






[23722 23778]


EXample result

[0 2 1 1 1 3]


In [10]:
print(np.unique(y_test))

#print all the unique values in array means no repeatation

#example
print("\n\nEXample result\n")
a=np.array([1,2,1,3,4,5,5,5])
print(np.unique(a))



[0 1]


EXample result

[1 2 3 4 5]


In [11]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression   #for predection
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import GridSearchCV #hyper-path

In [78]:
tfidf = TfidfVectorizer(stop_words='english')    #initialising TfidfVectorizer   

#this creates the matrix of tfids for each word in the input file

#stop_words='english is used to ignore common english word like 'the' 'a' which does not have meaning


#https://stackoverflow.com/questions/34449127/sklearn-tfidf-transformer-how-to-get-tf-idf-values-of-given-words-in-documen

#example

testtfidf=TfidfVectorizer(stop_words='english')

corpus = ["hotel food","hotel food service"]
testtfidf.fit_transform(corpus)
print(testtfidf.vocabulary_)

0 1 2 1 2 3


In [79]:

param_grid = {'clf__C':[1.0,10.0,100.0]}
X_train.ndim   #print the Number of array dimensions.

#here dimension means real world dimensions 1d 2d 3d or nd

#example is given below


0 1 2 1 2 3


In [80]:

a1 = np.array(111)
b1 = np.array([1,2])
c1 = np.array([[1,2], [4,5]])
d1 = np.array([[1,2,3,], [4,5,]])
e1=np.array([[1,2,3],[4,5,6],[7,8,9]])
f1=np.array([[[0, 0, 0], [0, 0, 0], [0, 0, 0]],
 [[0, 0, 0], [0, 0, 0], [0, 0, 0]],
 [[0, 0, 0], [0, 0, 0], [0, 0, 0]]])   #this is a 3d array
print(a1.ndim, b1.ndim, c1.ndim, d1.ndim, e1.ndim,f1.ndim)


0 1 2 1 2 3


In [56]:
Ir_tfidf = Pipeline([('vect',tfidf),('clf',LogisticRegression())])
#http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

#






gs_Ir_tfidf = GridSearchCV(Ir_tfidf,param_grid,scoring='accuracy')

In [57]:
gs_Ir_tfidf.fit(X_train,y_train)




GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__C': [1.0, 10.0, 100.0]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='accuracy',
       verbose=0)

In [58]:
print('Best parameter set: %s ' % gs_Ir_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_Ir_tfidf.best_score_)


Best parameter set: {'clf__C': 10.0} 
CV Accuracy: 0.829


In [59]:
clf = gs_Ir_tfidf.best_estimator_

In [60]:
clf.score(X_test,y_test)

0.8492842105263158

In [61]:
from sklearn.naive_bayes import MultinomialNB

In [62]:
nb = Pipeline([('vect', tfidf),('clf', MultinomialNB())])

In [63]:
nb.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [64]:
nb.score(X_train,y_train)

0.9576169532187125

In [65]:
nb.score(X_test,y_test)

0.806821052631579

In [ ]:


new_data.shape


In [47]:
nb = MultinomialNB()
nb.fit(new_data,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [48]:
nb.score(new_test,y_test)

0.8444842105263158

In [49]:
nb.score(new_data,y_train)

0.9812075169932027